In [1]:
import pandas as pd
train_data = pd.read_parquet(r"Data\UNSW_NB15_training-set.parquet", engine="pyarrow")
test_data = pd.read_parquet(r"Data\UNSW_NB15_testing-set.parquet", engine="pyarrow")

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

Training data shape: (175341, 36)
Testing data shape: (82332, 36)
